In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(7467, 128, 128, 3)

In [8]:
y.shape

(7467, 4)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(5973, 128, 128, 3)

In [11]:
X_test.shape

(1494, 128, 128, 3)

In [12]:
y_train.shape

(5973, 4)

In [13]:
y_test.shape

(1494, 4)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

4

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 14:18:05.530201: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


187/187 [==============================] - 115s 610ms/step - loss: 0.4324 - accuracy: 0.8386 - val_loss: 0.3544 - val_accuracy: 0.8795
Epoch 2/10
187/187 [==============================] - 145s 774ms/step - loss: 0.2120 - accuracy: 0.9248 - val_loss: 0.1553 - val_accuracy: 0.9471
Epoch 3/10
187/187 [==============================] - 140s 746ms/step - loss: 0.1398 - accuracy: 0.9523 - val_loss: 0.3375 - val_accuracy: 0.8909
Epoch 4/10
187/187 [==============================] - 136s 726ms/step - loss: 0.1023 - accuracy: 0.9645 - val_loss: 0.1407 - val_accuracy: 0.9478
Epoch 5/10
187/187 [==============================] - 142s 760ms/step - loss: 0.0912 - accuracy: 0.9689 - val_loss: 0.1295 - val_accuracy: 0.9578
Epoch 6/10
187/187 [==============================] - 134s 717ms/step - loss: 0.0740 - accuracy: 0.9761 - val_loss: 0.4660 - val_accuracy: 0.8487
Epoch 7/10
187/187 [==============================] - 129s 692ms/step - loss: 0.0548 - accuracy: 0.9829 - val_loss: 0.0608 - val_accura

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 98.73%


In [19]:
cnn_scores

[0.04595142975449562, 0.9872824549674988]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

47/47 [==============================] - 4s 85ms/step
Accuracy: 98.73%
Precision: 98.76%
Recall: 98.73%
F1 score: 98.73%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-peach+pepper.csv', index=False)

+------+-------------------------------------------------------------------------+-------------------------------+-------------------------------+
|      | Filename                                                                | True Labels                   | Predicted Labels              |
|------+-------------------------------------------------------------------------+-------------------------------+-------------------------------|
|    0 | eaef6117-c055-401a-8851-cec16787b5d8___JR_B.Spot 3278_flipTB.JPG        | Pepper,_bell___Bacterial_spot | Pepper,_bell___Bacterial_spot |
|    1 | 98558acc-09f4-4d2f-bdff-5e8e915ea606___NREC_B.Spot 9141.JPG             | Pepper,_bell___Bacterial_spot | Pepper,_bell___Bacterial_spot |
|    2 | 0e10ce50-cadb-403e-9d98-6e9b31bf7c24___Rutg._Bact.S 1791.JPG            | Peach___Bacterial_spot        | Peach___Bacterial_spot        |
|    3 | 46053143-6404-4496-8d26-9f801d1c6f4f___Rutg._HL 3532.JPG                | Peach___healthy               | Pea

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-peach+pepper.csv', index=False)

False Predictions:
+------+--------------------------------------------------------------+------------------------+-------------------------------+
|      | Filename                                                     | True Labels            | Predicted Labels              |
|------+--------------------------------------------------------------+------------------------+-------------------------------|
|  471 | b61a5e27-1da3-4c61-9951-691fdceb9e81___JR_HL 8744.JPG        | Pepper,_bell___healthy | Pepper,_bell___Bacterial_spot |
|  491 | 6cadb764-7b1d-42e9-b142-ece7fe4eec36___Rutg._Bact.S 1479.JPG | Peach___Bacterial_spot | Pepper,_bell___Bacterial_spot |
|  554 | d7f13145-ea6d-4e04-9efa-669792d5e056___JR_HL 5980.JPG        | Pepper,_bell___healthy | Pepper,_bell___Bacterial_spot |
|  620 | 1d97669b-b6a5-4239-a709-2b2c9de4d2d5___Rut._Bact.S 3449.JPG  | Peach___Bacterial_spot | Pepper,_bell___healthy        |
|  621 | e57adc99-0329-4dd4-96b7-0bb087c020cb___JR_HL 8706.JPG        | Pepper

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

187/187 [==============================] - 13s 68ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 28.201284885406494 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

47/47 [==============================] - 3s 68ms/step
Random Forest Classifier Accuracy: 98.39%
Random Forest Classifier Precision: 0.98
Random Forest Classifier Recall: 0.98
Random Forest Classifier F1 Score: 0.98


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-peach+pepper.csv', index=False)

+------+-------------------------------------------------------------------------+-------------------------------+-------------------------------+
|      | Filename                                                                | True Labels                   | Predicted Labels              |
|------+-------------------------------------------------------------------------+-------------------------------+-------------------------------|
|    0 | eaef6117-c055-401a-8851-cec16787b5d8___JR_B.Spot 3278_flipTB.JPG        | Pepper,_bell___Bacterial_spot | Pepper,_bell___Bacterial_spot |
|    1 | 98558acc-09f4-4d2f-bdff-5e8e915ea606___NREC_B.Spot 9141.JPG             | Pepper,_bell___Bacterial_spot | Pepper,_bell___Bacterial_spot |
|    2 | 0e10ce50-cadb-403e-9d98-6e9b31bf7c24___Rutg._Bact.S 1791.JPG            | Peach___Bacterial_spot        | Peach___Bacterial_spot        |
|    3 | 46053143-6404-4496-8d26-9f801d1c6f4f___Rutg._HL 3532.JPG                | Peach___healthy               | Pea

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-peach+pepper.csv', index=False)

False Predictions:
+------+---------------------------------------------------------------------+-------------------------------+-------------------------------+
|      | Filename                                                            | True Labels                   | Predicted Labels              |
|------+---------------------------------------------------------------------+-------------------------------+-------------------------------|
|  186 | 889676f3-e37b-4b86-a440-c087611e5cd5___Rutg._HL 2432_90deg.JPG      | Peach___healthy               | Pepper,_bell___healthy        |
|  259 | 018e494e-d2eb-468b-9d02-40219d9f4921___JR_B.Spot 9045.JPG           | Pepper,_bell___Bacterial_spot | Pepper,_bell___healthy        |
|  284 | cec0354e-5caf-4eb7-9fa5-a7b07ac1b644___JR_B.Spot 9070.JPG           | Pepper,_bell___Bacterial_spot | Pepper,_bell___healthy        |
|  472 | 75f876b6-2b3e-4b24-b337-6f7754e1df23___Rut._Bact.S 1291.JPG         | Peach___Bacterial_spot        | Pepper,_bell